Coursera의 Neural Networks for Machine Learning 강의를 정리한 내용임.   
2016.11.21. by Dongwan Kim

# Week7. Recurrent neural networks

### Lecture 7a, Modeling sequenes: A brief overview

어떤 순차적인 데이터(sequences)를 입력값으로 받아 출력값을 내놓는 시스템을 생각해 볼 수 있다. 출력값은 다른 sequence가 될 수도 있고 입력 sequence에 추가적인 정보가 포함된 sequence가 될 수도 있다. 예를들어 시간순의 sound pressure 데이터를 입력으로 받아 그 소리가 나타내는 단어들을 출력으로 내놓을 수도 있고, $t-2$와 $t-1$시점의 단어를 입력받아 $t$시점의 단어를 예측하여 추가로 출력으로 내놓을 수도 있을 것이다.

별도의 target sequence가 없는 경우도 있는데 예를들어 문장을 자동으로 생성하는 시스템의 경우 다음 단어에 대한 특별한 정답이 없고, 출력값을 다시 입력값으로 사용하여 다다음 단어를 생성해 가는 과정을 반복하게 된다. 따라서 이와 같이 sequential한 예측 문제의 경우 supervised learning과 unsupervised learning 사이의 경계가 불분명하다.

RNN을 설명하기 전에 대표적인 generative model 두가지를 짚고 넢어가기로 한다. 첫번째로 **Linear Dynamical System**이 있는데 hidden state이 real value이고, Gaussian noise(linear dynamics)를 갖는 것이 특징이다.

<img  src="./_images/07_LDS.PNG" title="Linear Dynamical System"/>

> **Continuous-time linear dynamical system(CT LDS)**은 아래와 같은 형태를 갖으며 첫번째 식은 dynamics equation, 두번째 식은 measurement equation 또는 readout equation이라 한다.
$$\begin{align}
\frac{dx}{dt}&=A(t)x(t) + B(t)u(t) \\
y(t) &= C(t)x(t) + D(t)u(t)
\end{align}$$
- $t \in \mathbf{R}$ : time
- $x(t) \in \mathbf{R}^n$ : state(vector)
- $u(t) \in \mathbf{R}^m$ : input or control
- $y(t) \in \mathbf{R}^p$ : output
- $A(t) \in \mathbf{R}^{n \times n}$ : dynamics matrix
- $B(t) \in \mathbf{R}^{n \times m}$ : input matrix
- $C(t) \in \mathbf{R}^{p \times n}$ : output or sensor matrix
- $D(t) \in \mathbf{R}^{p \times m}$ : feedthrough matrix

> 위 두 식을 아래와 같이 표현하기도 한다.
$$\begin{align}
\dot{x}&=Ax + Bu \\
y &= Cx + Du
\end{align}$$

> - 대부분의 linear system은 time-invariant하다. 즉 $A$, $B$, $C$, $D$가 시간에 따라 변하지 않는다.(상수)
> - 입력값 $u$가 없는 경우(따라서 물론 $B$나 $D$도 없다.) autonomous system이라 한다.
> - 종종 $D=0$경우도 있다.
> - $u(t)$와 $y(t)$가 scalar인 경우 이런 시스템을 single-input, single-output(SISO)라 한다. 반대로 scalar가 아닌 경우 MIMO라 한다.

---

> **Discrete-time linear dynamical system(DT LDS)**는 아래와 같은 형태를 갖는다.
$$\begin{align}
x(t+1) &= A(t)x(t) + B(t)u(t) \\
y(t) &= C(t)x(t) + D(t)u(t)
\end{align}$$

> - $t \in \mathbf{Z} = \{ 0, \pm1, \pm2, \dots \}$ : time, period, sample #...
> - $x$, $u$, $y$ : (vector) sequence

---
> Linear system의 활용 예는 아래와 같다.
- automatic control system : 1960년대~ 로켓/우주선 컨트롤
- signal processing : 1990년대~ 신호처리
- communications : 2000년대~ 통신
- economics, finance
- circuit analysis, simulation, design
- mechanical and civil engineering
- aeronautics
- navigation, guidance

---
> Reference
 - https://www.youtube.com/watch?v=bf1264iFr-w&t=258s
 - http://mathinsight.org/solving_linear_discrete_dynamical_system_examples

다음으로 **HMM(Hidden Markov Models)**가 있는데, hidden state이 $N$개 중 하나의 state을 갖는 것이 특징이다. output을 통해 각 시점의 hidden state에 대한 discrete probability distribution을 추론할 수 있고, 각 시점간의 state변환 또한 stochastic하게 진행된다. 물론 output 생성 또한 stochastic하다.

HMM은 각 시점에 $N$개 중 하나의 상태값을 갖는데, 이는 $\log N$개의 bit들을 의미한다.($N = 2^{\text{num of bits}}$), 즉 이전 시점의 정보 중 $\log N$개의 bit만을 기억하게 된다. 따라서 기억해야 하는 정보의 량이 많을 경우 state의 수가 exponential하게 많아야 하는 단점이 있다.

<img  src="./_images/07_HMM.PNG" title="HMM"/>

그렇다면 RNN은 어떠한가? 우선 앞서 두 방법(LDS, HMM)에 비해 훨신 많은 정보를 기억시킬 수 있고, non-linear dynamics를 이용하여 보다 복잡한 형태로 hidden state을 학습시킬 수 있는 장점이 있다.

<img  src="./_images/07_RNN.PNG"/>

Linear dynamical system과 hidden Markov model 모두 stochastic model이다. 즉 hidden state의 전이가 확률적으로 변화하고 hidden state이 생성하는 output 또한 확률적이다. 하지만 관측 데이터가 주어졌을 때 hidden state(over states)의 posterior probability distribution은 deterministic function이다. 그런데 RNN의 hidden state또한 어떤 probability distribution이라 생각할 수 있고 이 또한 deterministic function이기 때문에 LDS, HMM과의 유사성을 생각해 볼 수 있다.



### Lecture 7b, Training RNNs with backpropagation

RNN을 학습시키는 대표적인 방법인 **BPTT(Back-Propagation Through Time) algorithm**에 대해 알아보자.

아래와 같이 좌측의 RNN을 각 (discrete)time step이 하나의 layer이고, 각 layer간의 weight은 반복적으로 사용하는 형태의 우측 feedforward net으로 표현할 수 있다. (<u>layered feed-forward net with shared weights</u>)


<img  src="./_images/07_RNN_ffnn.PNG"/>

이렇게 feedforward net으로 표현하고나서 backpropagation으로 weight을 학습시킨다는 아이디어가 **BPTT(Back-Propagation Through Time) algorithm**이다. 다만 각 time step의 $w_i$가 같다는 constraint가 있으므로 tied weight들의 weight gradient는 각 weight gradient의 평균을 사용하게 된다.


이런 접근 방법을 사용할 때 좀 골치아픈 문제가 몇가지 있는데, 우선 각 unit의 초기값을 설정하는 문제이다. 물론 어떤 임의의 값으로 설정하고 초기값 또한 parameter로 취급하여 학습시키는 방법을 사용할 수도 있다.

이런 순환 구조의 Neural Network에 input value를 할당하는 방법이 여러가지 있는데, 아래 그림과 같이 특정 time step의 전체 unit의 state에 input value를 할당하는 방법과 모든 time step의 일부 unit의 state에 input value를 할당하는 방법이 있다. 후자의 방법은 sequential data를 입력할때 좋은 방법이라 할 수 있다.(특정 unit에 time step마다 값을 할당하면 되므로)

<img  src="./_images/07_RNN_input1.PNG"/>

target value를 할당하는 방법도 있는데, 마지막 몇개의 time step의 전체 unit에 할당하는 방법과 전체 time step의 일부 unit에 할당하는 방법이 있다.

<img  src="./_images/07_RNN_teaching_signal.PNG"/>

### Lecture 7c, A toy example of training an RNN

RNN을 활용하는 간단한 예로 binary addition을 살펴보자. 예를들어 input bits [00100110]과 [10100110]을 입력으로 받아 [11001100]을 출력으로 내놓는 시스템을 생각해 볼 수 있는데, 이런 문제를 아래와 같은 feedforward neural net으로도 설계할 수 있을 것이다. 하지만 feedforward neural net으로 만들 경우 input과 output의 bit수를 미리 정해야 하는 단점이 있다.

<img  src="./_images/07_binary_addition_ffnn.PNG"/>

사실 이 문제를 finite state automaton(https://en.wikipedia.org/wiki/Finite-state_machine)으로 아래와 같이 모델링 할수 있다. 시스템은 특정 time step에 [no carry print1], [carry print1], [no carry print0], [carry print0]의 4가지 상태 중 하나의 상태만을 갖는다. 각 상태에서 들어온 입력값에 따라 4가지 상태중 하나로 전이(trainsition)하게 된다.

예를들어 우측 상단의 [carry print1]상태에서 [1,1]을 입력으로 받게 되면 1을 다음 자리로 넘기고 1이 결과값 이므로 [carry print1]상태에 그대로 머물게 된다. 반면 [1,0]이나 [0, 1]을 입력으로 받게 되면 1을 다음자리로 남겨야 하고 결과값을 0 이므로 [carry print0] 상태로 전이하게 된다.

이 모델은 HMM과 상당히 유사한데, 다만 각 상태가 hidden이 아니라는(no dynamics) 차이가 있다.

<img  src="./_images/07_binary_addition_fsa.PNG"/>

이 문제를 아래 그림과 같이 RNN으로 모델링 할 수도 있다. 두개의 input unit과 fully interconnected 3 hidden unit, 그리고 하나의 output unit으로 구성된 형태이다.

<img  src="./_images/07_RNN-Arithmetic.png"/>
<h6 align='right'>http://devankuleindiren.com/Projects/rnn_arithmetic.php</h6>

### Lecture 7d, Why it is difficult to train an RNN

RNN을 학습을 어렵게 하는 것으로 exploding or vanishing gradient 문제를 소개한다. RNN의 time step이 많아진다는 것은 feed-forward net의 hidden layer가 많아지는 것과 같다. 그런데 hidden layer가 한두개 일때에는 문제가 되지 않지만 hidden layer가 많아질 경우 back-propagation이 진행되면서 gradient값이 지나치게 작아지거나 지나치게 커져버리는 일일 발생하게 된다.

이런 현상의 원인은 backward pass의 linear한 특성 때문인데, 예를들어 logistic unit을 사용하는 NN에서 forward pass의 경우 아래 그림에서 파란선과 같이 각 unit에 들어오는 입력값(weighted sum)에 따라 0에서 1사이의 값을 출력하게 된다. 이 때문에 forward pass의 출력값에는 exploding 문제가 발생하지 않는다. 반면 backward pass의 경우 각 unit의 activation에 대한 input의 편미분값($\frac{\partial \mathbf{A}}{\partial \mathbf{Z}}$)이 forward pass때 지나갔던 (빨간)점에서의 기울기이므로 만약 (sigmoid activation을 사용했을 때) activation 값이 작아져 0에 가깝다면 그때의 gradient 값도 0이 되어 이어지는 back propagation도 0에 가까워지는 악순환이 반복된다. 즉 back propagation이 뒤로 갈수록 미미해지게 된다.

- 0에서의 saturation을 줄이기 위해 tanh 함수를 activation function으로 사용하는 방법도 있으며 자세한 내용은 
 - <a href='http://machinelearning.wustl.edu/mlpapers/paper_files/AISTATS2010_GlorotB10.pdf' target='_blank'>Understanding the difficulty of training deep feedforward neural networks
</a>나 
 - <a href='https://www.quora.com/What-are-the-benefits-of-a-tanh-activation-function-over-a-standard-sigmoid-activation-function-for-artificial-neural-nets-and-vice-versa' target='_blank'>What are the benefits of a tanh activation function over a standard sigmoid</a> 를 참고

<img  src="./_images/07_RNN_backprop.PNG"/>

좀더 formal 하게 예기해보면.. $n$개의 layer로 구성된 feed-forward network를 생각해 보자. $i$번째 layer는 $j$개의 unit으로 구성되어 있고, 이 layer의 unit들은 $\mathbf{Z}_i \in \mathbb{R}^j$라는 input sequence(matrix)을 받아, $\sigma()$라는 activation function을 거쳐, $\mathbf{A}_i \in \mathbb{R}^j$라는 output sequence를 내 놓는다고 해 보자. 또한 $i-1$번째 layer와 $i$번째 layer를 연결하는 weight matrix를 $\mathbf{W}_{(i, i-1)} \in \mathbb{R}^{(i-1) \times i}$이라 하고(week5 PA2에서의 weight matrix와는 달리 row/column을 뒤바꿈) $i$번째 layer의 input bias를 $\mathbf{b}_i$라 하면 $\mathbf{Z}_i$는 $\mathbf{W}_{(i, i-1)}^T  \mathbf{\mathbf{A}_{i-1}} + \mathbf{b}_i$와 같다. 마지막 output layer에서 계산된 cost sequence를 $\mathbf{E}$라 해 보자.

이때 마지막 $n$번째 layer부터 각 layer 사이의 weight gradient를 생각해 보자. 아래와 같이 마지막 layer부터 derivative값을 곱하고 (matrix 내에서)더하는 과정이 반복된다. 즉 weighted sum의 linear operation만으로 weight들을 update 하게 된다.


$$\begin{align} \frac{\partial \mathbf{E}}{\partial \mathbf{W}_{(i, i-1)}} &= \frac{\partial \mathbf{Z}_n}{\partial \mathbf{W}_{(i, i-1)}} \frac{\partial \mathbf{E}}{\partial \mathbf{Z}_n} \\
\frac{\partial \mathbf{E}}{\partial \mathbf{W}_{(i-1, i-2)}} &= \frac{\partial \mathbf{Z}_{i-1}}{\partial \mathbf{W}_{(i-1, i-2)}} \frac{\partial \mathbf{A}_{i-1}}{\partial \mathbf{Z}_{i-1}} \frac{\partial \mathbf{Z}_n}{\partial \mathbf{A}_{i-1}} \frac{\partial \mathbf{E}}{\partial \mathbf{Z}_n} \\
\frac{\partial \mathbf{E}}{\partial \mathbf{W}_{(i-2, i-3)}} &= \frac{\partial \mathbf{Z}_{i-2}}{\partial \mathbf{W}_{(i-2, i-3)}} \frac{\partial \mathbf{A}_{i-2}}{\partial \mathbf{Z}_{i-2}} \frac{\partial \mathbf{Z}_{i-1}}{\partial \mathbf{A}_{i-2}} \frac{\partial \mathbf{A}_{i-1}}{\partial \mathbf{Z}_{i-1}} \frac{\partial \mathbf{Z}_n}{\partial \mathbf{A}_{i-1}} \frac{\partial \mathbf{E}}{\partial \mathbf{Z}_n} \\
\frac{\partial \mathbf{E}}{\partial \mathbf{W}_{(i-3, i-4)}} &= \frac{\partial \mathbf{Z}_{i-3}}{\partial \mathbf{W}_{(i-3, i-4)}} \frac{\partial \mathbf{A}_{i-3}}{\partial \mathbf{Z}_{i-3}} \frac{\partial \mathbf{Z}_{i-2}}{\partial \mathbf{A}_{i-3}} \frac{\partial \mathbf{Z}_{i-2}}{\partial \mathbf{A}_{i-3}} \frac{\partial \mathbf{A}_{i-2}}{\partial \mathbf{Z}_{i-2}} \frac{\partial \mathbf{Z}_{i-1}}{\partial \mathbf{A}_{i-2}} \frac{\partial \mathbf{A}_{i-1}}{\partial \mathbf{Z}_{i-1}} \frac{\partial \mathbf{Z}_n}{\partial \mathbf{A}_{i-1}} \frac{\partial \mathbf{E}}{\partial \mathbf{Z}_n}
\end{align}
$$

이와 같이 backward pass의 linear한 특성 때문에 weight들이 작다면 gradient값도 exponential하게 작아지고(vanishing gradient), weight들이 크다면 그 반대의 현상(exploding gradient)이 발생한다. 이런 문제를 해결하기 위한 방법으로 weight들의 초기값을 매우 잘 할당하는 방법이 있고, 어느정도 효과가 있다. 하지만 잘 초기화 했다 하더라도 입력값이 출력값에 어떤 영향을 주는지 알기가 쉽지 않다.

---

exploding or dying gradient의 예로 input sequence에 대한 attractor를 학습하는 모델을 생각 해 보자. 
> attractor는 아주 간단히 말하면, 어떤 영역(basin)에서 출발하면 그 시작점이 다르더라도 시간이 지나면 결국 모이게 되는 안정된 궤도 혹은 지점을 말한다. 예를들어 서로 다른 위치에서 위성을 쏳아올리더라도 결국 어떤 목표한 궤도를 돌게 되는데, 이 궤도를 attractor(끌개)라고 한다.(https://en.wikipedia.org/wiki/Attractor)

아래 그림에는 파란색과 빨간색의 두개의 끌림 영역(basin of attractor)이 존재하고, 파란 영역의 적당한 점에서 궤도를 시작하면 결국은 1번 점에 모이게 된다. 반면 빨간 영역의 작당한 점에서 궤도를 시작하면 결국 2번 점에 모이게 된다. 따라서 시작점의 차이는 시간이 자날수록 상쇄(vanished)된다. 반면 빨간 영역과 파란 영역의 경계부근에서는 흥미로운 일이 일어난다. 즉 약간의 시작점 차이에 따라 1번점 혹은 2번 점으로 그 결과가 명백히 갈리게 된다. 즉 초기값에 따라 아주 큰 결과값 차이가 발생(exploded)하게 된다. 

<img  src="./_images/07_attractor2.PNG"/>

앞서 설명한 것 처럼 RNN을 학습시키는 것은 쉬운 일이 아니데, 학습을 좀더 효과적으로 할 수 있는 몇가지 방법이 있다.
- Long Short Term Memory: Lecture 7e
- Hessian Free Optimization: Lecture 8a
- Echo State Networks: Lecture 8d
- Good initialization with momentum

### Lecture 7e, Long-term Short-term-memory

Neural network의 dynamic state을 short term memory라 할 수 있는데, 이 short term memory를 좀 더 오래 지속되도록 하겠다는 아이디어가 바로 **Long-term Short-term-memory(LSTM)**이다.

unit의 state값을 오래도록 지속시키기 위해 'write gate', 'keep gate', 'read gate'라는 것을 사용 했다. 아래 그림에서 가운데 unit이 memory cell인데 현재 1.73을 상태값으로 기억하고 있다. 그리고 주변의 logistic unit인 write gate의 값이 1에 가까울 때에만 상태값 입력이 발생하고, 마찬가지로 logistic unit인 keep gate의 activation이 1에 가까울 때에는 memory cell의 값이 변경되지 않는다. 마지막으로 read gate이 켜지면 값을 읽게 된다. (실제로는 keep gate이 1에 가까울 때에는 memory cell의 값을 각 time step마다 같은 값을 입력하는 일이 반복된다.)

<img  src="./_images/07_RNN_lstm.PNG"/>

Graves & Schmidhuber (2009)에서 LSTM을 사용하는 RNN으로 필기체(cursive writing)을 인식하는 시스템이 소개된다. 꽤 성능이 좋았는데.. 아래 그림과 같이 동작한다. 첫줄에는 인식된 문자, 둘째줄에는 memory cell에 기억된 값들, 세번째 줄에는 실제 필기체, 네번째 줄에는 결정에 가장 영향력 있는 unit들의 backpropagated gradient값을 보여주고 있다.

<img  src="_images\07_RNN_handwriting_graves.PNG"/>

---

번외로 간단한 RNN예제를 살펴보자.

<img  src="./_images/07_RNN_example2.PNG"/>

좌측부터 각각 time step $t=0, 1, 2, 3, 4$에서의 RNN을 unroll 해 놓았고, input unit $x$, output unit $y$, logistic($\sigma(z)$) hidden unit $h$로 구성되어 있다. 이때 각 output 값 $y_0, y_1, y_2, y_3$은 아래와 같다.

$$\begin{align}
a_{h0} &= \sigma(w_{xh} x_0 + bias_h) \\
y_0 &= w_{hy} ~ a_{h0} + bias_y \\
\\
a_{h1} &= \sigma(w_{xh} x_1 + bias_h + w_{hh} a_{h0}) \\
y_1 &= w_{hy} ~ a_{h1} + bias_y \\
\\
a_{h2} &= \sigma(w_{xh} x_2 + bias_h + w_{hh} a_{h1}) \\
y_2 &= w_{hy} ~ a_{h2} + bias_y \\
\\
a_{h3} &= \sigma(w_{xh} x_3 + bias_h + w_{hh} a_{h2}) \\
y_3 &= w_{hy} ~ a_{h3} + bias_y \\
\end{align}
$$